In [1]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.5 MB/s eta 0:00:00


## Yolo compression

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torchvision import models
from torch.nn.utils import prune
from torch.utils.data import DataLoader
from ultralytics import YOLO

import time
from tqdm.notebook import tqdm

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
model = YOLO('yolov8l.pt')

100%|██████████| 83.7M/83.7M [00:00<00:00, 215MB/s]


In [4]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 166.848MB


In [5]:
model.eval()
!yolo task=detect mode=val model=yolov8l.pt name=yolov8l_eval

WARNING ⚠️ 'data' argument is missing. Using default 'data=coco8.yaml'.
Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8l summary (fused): 112 layers, 43,668,288 parameters, 0 gradients, 165.2 GFLOPs

WARNING ⚠️ Dataset 'coco8.yaml' images not found, missing path '/content/datasets/coco8/images/val'
100% 433k/433k [00:00<00:00, 16.8MB/s]
Unzipping /content/datasets/coco8.zip to /content/datasets/coco8...: 100% 25/25 [00:00<00:00, 4987.99file/s]
Dataset download success ✅ (0.6s), saved to /content/datasets

100% 755k/755k [00:00<00:00, 21.5MB/s]
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1631.7±719.7 MB/s, size: 54.0 KB)
val: Scanning /content/datasets/coco8/labels/val... 4 images, 0 backgrounds, 0 corrupt: 100% 4/4 [00:00<00:00, 131.97it/s]
val: New cache created: /content/datasets/coco8/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:01<00:00,  1.71s/it]
            

In [6]:
inp = torch.randn(1, 3, 224, 224)

num_samples = 1000
start_time = time.time()
for _ in tqdm(range(num_samples)):
    output = model((inp / 255), verbose=False)
end_time = time.time()

infer_time = ((end_time - start_time) / num_samples) * 1000
print("---"*10, f'Avg inference time: {infer_time:.4f} ms', sep='\n')

  0%|          | 0/1000 [00:00<?, ?it/s]

------------------------------
Avg inference time: 14.0131 ms


### Квантизация

In [7]:
model = YOLO('yolov8l.pt')

In [8]:
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Conv2d},
    dtype=torch.qint8
)

torch.save(quantized_model.state_dict(), 'quantized.pt')

In [9]:
param_size = 0
for param in quantized_model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in quantized_model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('quantized_model size: {:.3f}MB'.format(size_all_mb))

quantized_model size: 166.848MB


In [10]:
inp = torch.randn(1, 3, 224, 224)

num_samples = 1000
start_time = time.time()
for _ in tqdm(range(num_samples)):
    output = quantized_model((inp / 255), verbose=False)
end_time = time.time()

infer_time = ((end_time - start_time) / num_samples) * 1000
print("---"*10, f'Avg inference time: {infer_time:.4f} ms', sep='\n')

  0%|          | 0/1000 [00:00<?, ?it/s]

------------------------------
Avg inference time: 13.2802 ms


### Прунинг

In [11]:
model = YOLO('yolov8l.pt')

In [12]:
for name, m in model.named_modules():
  params = list(m.named_parameters())
  if len(params) and params[0][0] == 'weight':
    prune.l1_unstructured(m, name=params[0][0], amount=0.3)

In [13]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 333.428MB


In [14]:
inp = torch.randn(1, 3, 224, 224)

num_samples = 1000
start_time = time.time()
for _ in tqdm(range(num_samples)):
    output = model((inp / 255), verbose=False)
end_time = time.time()

infer_time = ((end_time - start_time) / num_samples) * 1000
print("---"*10, f'Avg inference time: {infer_time:.4f} ms', sep='\n')

  0%|          | 0/1000 [00:00<?, ?it/s]

------------------------------
Avg inference time: 13.7457 ms
